In [3]:
using DifferentialEquations
using Plots
using DataFrames
using Peaks #For detecting oscillations
using Random
using Polyhedra #For creating convex hulls
using QHull #For creating convex hulls
using Surrogates
using AbstractGPs
using SurrogatesAbstractGPs

In [7]:
#Define ODE model and solve
"""ode(du, u, t)
        Defines a system of differential equations for a 2D regulon with multiple steady states 
""" 
function ode(du, u, p, t)
    du[1] = (1.95*u[1]^3)/(1+u[1]^3) + (9*u[2]^3)/(6^3 + u[2]^3) - u[1]
    du[2] = (10*u[1]^3)/(7^3 + u[1]^3) + (1.95*u[2]^3)/(1+u[2]^3) - u[2]
end

"""solve_ode(u0)
        Integrates ODE system and returns the solution.
""" 
function solve_ode(u0, endtime=1E4, resolution=200)
    tspan = [0, endtime] #Span of time to integrate for
    savetimes = LinRange(0, endtime, resolution) #linear space of points until final time
    p = []
    prob = ODEProblem(ode, u0, tspan, p)
    sol = solve(prob, Tsit5(), reltol=1e-3, abstol=1e-6, saveat=savetimes)
    return sol
end 

#Sample parameter space
"""initial_sampling(lower_bounds, upper_bounds, n_init)
        Returns n_init samples between lower_bounds and upper_bounds. 
""" 
function initial_sampling(lower_bounds, upper_bounds, n_init)
    #Sample uniform points across space
    n_dims = length(lower_bounds)
    samples = Matrix{Float64}(undef, n_init, n_dims)
    for d in 1:n_dims
        lb = lower_bounds[d]
        ub = upper_bounds[d]
        for i in 1:n_init
            samples[i, d] = rand() * (ub - lb) + lb
        end
    end
    return samples
end


function class_name(ss)
    return string(round(abs(ss[1]), digits=1))*", "*string(round(abs(ss[2]), digits=1))
end

function compute_classes(samples)
    return [class_name(solve_ode(samples[i, :]).u[end]) for i in 1:size(samples)[1]]
end

compute_classes (generic function with 1 method)

In [9]:
#Generate uniform intial points and classify
n_init = 25
samples = initial_sampling([0, 0], [10, 10], n_init)
classes = compute_classes(samples)
#Compute centroid of each class of points

25-element Vector{String}:
 "8.5, 8.4"
 "8.5, 8.4"
 "2.1, 2.0"
 "2.1, 2.0"
 "8.5, 8.4"
 "2.1, 2.0"
 "8.5, 8.4"
 "2.1, 2.0"
 "8.5, 8.4"
 "8.5, 8.4"
 ⋮
 "8.5, 8.4"
 "2.1, 2.0"
 "2.1, 2.0"
 "2.1, 2.0"
 "2.1, 2.0"
 "8.5, 8.4"
 "2.1, 2.0"
 "2.1, 2.0"
 "2.1, 2.0"